In [ ]:
## QICK LIBRARY
from qick import *

from random import random
import time
import pickle
import numpy as np

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger

##### Load FGPA BitStream

##################
soc = QickSoc('./qick_216_seb.bit')

In [ ]:
print(soc)

In [ ]:
def get_mode_code(outsel = 'product', mode = 'oneshot', stdysel='zero', phrst = 0):
    outsel_reg = {'product': 0, 'dds': 1, 'input': 2, 'zero': 3}[outsel]
    mode_reg = {'oneshot': 0, 'periodic': 1}[mode]
    stdysel_reg = {'last': 0, 'zero': 1}[stdysel]

    mc = phrst*0b10000+stdysel_reg*0b01000+mode_reg*0b00100+outsel_reg
    return mc 

# Generate 50 MHz tone on ch=0 -- DAC tile 0, blk 0 -> 0_228

In [ ]:
soc.tproc.tproc_cfg = 1024


print('---------------------------------------------')
print('Loading WaveParam Memory:')


ch = 0
freq = 50 # MHz
gain = 20000
outsel = 'dds'
mode = 'periodic'
phrst = 0

wr_buff = []
f = soc.freq2reg(freq, gen_ch=ch)
p = 0
g = gain
e = 0
l = soc.us2cycles(1, gen_ch=ch)
c = get_mode_code(outsel = outsel, mode = mode, phrst = phrst)
wr_buff.append( [f, p,e,g,l,c,0,0] )

## load_mem(mem_sel, buff_in, addr):
soc.tproc.load_mem(3, wr_buff, 0)

print('---------------------------------------------')
print("Reading WaveParam Memory:")
#read_mem(mem_sel, addr, length):
rd_buff = soc.tproc.read_mem(3, 0, 100)
print(rd_buff)

print('---------------------------------------------')

In [ ]:

asm = """
TEST -op(tuser -#307)
JUMP HERE -if(S) -op(tuser - #307) -uf
JUMP STATE_S0

// STATES
STATE_S0:
  WPORT_WR p%s wmem [&0] @0
  TIME inc_ref #3072
  JUMP END

END:
  JUMP END
""" % ch


p_txt, p_bin = Assembler.str_asm2bin(asm)
# # p_txt is the OP_CODE of the program
# print(np.array(p_txt))
# # p_bin is the binary data to load in the Memory
# print(np.array(p_bin))
soc.tproc.Load_PMEM(p_bin)


print('--- LOADING MEM')
soc.tproc.Load_PMEM(p_bin)

print('--- START THE PROCESSOR')
soc.tproc.start()

# Repeat above code but change to ch=6 -- DAC tile 3, blk 0 -> 0_231

Adjusting the channel everything works as expected.

In [ ]:
soc.tproc.tproc_cfg = 1024

print('---------------------------------------------')
print('Loading WaveParam Memory:')
ch = 6
freq = 50 # MHz
gain = 20000
outsel = 'dds'
mode = 'periodic'
phrst = 0

wr_buff = []
f = soc.freq2reg(freq, gen_ch=ch)
p = 0
g = gain
e = 0
l = soc.us2cycles(1, gen_ch=ch)
c = get_mode_code(outsel = outsel, mode = mode, phrst = phrst)
wr_buff.append( [f, p,e,g,l,c,0,0] )

## load_mem(mem_sel, buff_in, addr):
soc.tproc.load_mem(3, wr_buff, 0)

print('---------------------------------------------')
print("Reading WaveParam Memory:")
#read_mem(mem_sel, addr, length):
rd_buff = soc.tproc.read_mem(3, 0, 1)
print(rd_buff)

print('---------------------------------------------')

In [ ]:

asm = """
TEST -op(tuser -#307)
JUMP HERE -if(S) -op(tuser - #307) -uf
JUMP STATE_S0

// STATES
STATE_S0:
  WPORT_WR p4 wmem [&0] @0
  WPORT_WR p6 wmem [&0] @0
  TIME inc_ref #3072
  JUMP END

END:
  JUMP END
"""


p_txt, p_bin = Assembler.str_asm2bin(asm)
# # p_txt is the OP_CODE of the program
# print(np.array(p_txt))
# # p_bin is the binary data to load in the Memory
# print(np.array(p_bin))
soc.tproc.Load_PMEM(p_bin)


print('--- LOADING MEM')
soc.tproc.Load_PMEM(p_bin)

print('--- START THE PROCESSOR')
soc.tproc.start()

In [ ]:
print(wr_buff)

In [ ]:
print(soc.freq2reg(freq, gen_ch=6))
print(soc.freq2reg(freq, gen_ch=4))


In [ ]:
soc.tproc.tproc_cfg = 1024

print('---------------------------------------------')
print('Loading WaveParam Memory:')
freq = 75 # MHz
gain = 20000
outsel = 'dds'
mode = 'periodic'
phrst = 0

wr_buff = []
p = 0
g = gain
e = 0
c = get_mode_code(outsel = outsel, mode = mode, phrst = phrst)

ch = 4
f = soc.freq2reg(freq, gen_ch=ch)
l = soc.us2cycles(1, gen_ch=ch)
wr_buff.append( [f, p,e,g,l,c,0,0] )

ch = 6
f = soc.freq2reg(freq, gen_ch=ch)
l = soc.us2cycles(1, gen_ch=ch)
wr_buff.append( [f, p,e,g,l,c,0,0] )

## load_mem(mem_sel, buff_in, addr):
soc.tproc.load_mem(3, wr_buff, 0)

print('---------------------------------------------')
print("Reading WaveParam Memory:")
#read_mem(mem_sel, addr, length):
rd_buff = soc.tproc.read_mem(3, 0, 4)
print(rd_buff)
print('---------------------------------------------')


asm = """
TEST -op(tuser -#307)
JUMP HERE -if(S) -op(tuser - #307) -uf
JUMP STATE_S0

// STATES
STATE_S0:
  WPORT_WR p4 wmem [&0] @0
  WPORT_WR p6 wmem [&1] @0
  TIME inc_ref #3072
  JUMP END

END:
  JUMP END
"""

p_txt, p_bin = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)


print('--- LOADING MEM')
soc.tproc.Load_PMEM(p_bin)

print('--- START THE PROCESSOR')
soc.tproc.start()


In [ ]:
soc.set_mixer_freq(4, 0)

In [ ]:
soc.tproc.get_proc_status()